# Facility Location with patterns

In this task you need to solve the Facility Location Problem using patterns. 
You are given the following data:

* a range $I$ containing the potential locations where a facility can be opened
* a range $J$ containing the customers
* a tuple $b$ containing the customer demands $b_{j} \in \mathbb{Z}_{+}$ for every $j \in J$
* a tuple $f$ containing the opening costs $f_{i} \in \mathbb{Z}_{+}$ for each location $i \in I$
* a tuple $C$ containing the capacities $C_{i} \in \mathbb{Z}_{+}$ for each location $i \in I$
* a dictionary $c$ containing the connection costs $c_{i, j}$ for connecting facitily $i \in I$ with customer $j \in J$
* a tuple $P$ containing the power set of $J$

Your task is to find a feasible assignment of customers to facilities of minimum total cost, such that:

* every customer is assigned to exactly one opened facility
* total demand of all customers assigned to a facility $i \in I$ is at most its capacity $C_{i}$

The binary variable $l_{i, p}$ equals to $1$ if the subset of customers $p \subseteq J$ is assigned to the facility $i \in I$, otherwise $l_{i, p}$ equals to $0$.

For facility $i \in I$ let $\mathscr{P}^{i}$ the set of all "patterns", i.e. $\mathscr{P}^{i}$ equals the power set of $J$. We define the costs $c_{P}^{i}$ of pattern $P \in \mathscr{P}^{i}$ for facility $i \in I$ by
$$c_{P}^{i} := \left\{\begin{array}{ll} f_{i} + \sum_{j \in P}{c_{ij}}, & \text{if } P \not = \emptyset \\
         0, & \text{if } P = \emptyset \end{array}\right. .$$


In [11]:
from gurobipy import *
from itertools import chain, combinations

In [12]:
# The set of potential locations where a facility can be opened
I = range(3)

# The set of customers
J = range(5)

# Customer demands
b = (5, 7, 10, 14, 11)

# Opening costs
f = (5, 11, 9)

# Facility capacities
C = (10, 20, 20)

# Connection costs
c = {(0, 0): 2, (0, 1): 7, (0, 2): 7, (0, 3): 8, (0, 4): 8, (1, 0): 7,
  (1, 1): 2, (1, 2): 4, (1, 3): 7, (1, 4): 2, (2, 0): 2, (2, 1): 2,
  (2, 2): 2, (2, 3): 4, (2, 4): 8}

# The set of all patterns
P = tuple(chain.from_iterable(combinations(J, r) for r in range(len(J)+1)))

# Sanity checks
assert(len(J) == len(b))
assert(len(I) == len(f) == len(C))
assert(len(c) == len(I) * len(J))

In [13]:
model = Model('FacilityLocation')

In [14]:
l = {}
for i in I:
  for p in P:
    l[i, p] = model.addVar(vtype=GRB.BINARY, name=f'l[{i},{p}]')

In [15]:
cost = {}
for i in I:
  for p in P:
    if len(p) == 0:
      cost[i, p] = 0
    else:
      cost[i, p] = f[i] + quicksum(c[i, j] for j in p)

In [16]:
# set objective function
model.setObjective(quicksum(cost[i, p] * l[i, p] for i in I for p in P), GRB.MINIMIZE)

In [17]:
# add constraints 
# for facility i in I we exclude the non-feasible pattern, i.e. subsets of customers that cannot be assigned to it
# at the same time
for i in I:
  for p in P:
    if sum(b[j] for j in list(p)) > C[i]:
      model.addConstr(l[i, p] == 0)

In [18]:
# for each facility i in I, choose exactly ONE pattern
for i in I:
  model.addConstr(quicksum(l[i, p] for p in P) == 1)

In [19]:
# for each customer j in J, choose exactly ONE pattern that asssigns them to a facility
for j in J:
  model.addConstr(quicksum(l[i, p] for i in I for p in P if j in list(p)) == 1)

In [20]:
model.update()
model.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 76 rows, 96 columns and 404 nonzeros
Model fingerprint: 0x93cc4f12
Variable types: 0 continuous, 96 integer (96 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+00, 4e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 76 rows and 96 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.05 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: 42 

Optimal solution found (tolerance 1.00e-04)
Best objective 4.200000000000e+01, best bound 4.200000000000e+01, gap 0.0000%
